In [ ]:
import MagmaPandas.geochemistry.volatile_solubility.IaconoMarziano as im
import MagmaPandas.geochemistry.volatile_solubility.Allison2022 as a
im_config = im.IaconoMarziano_configuration
im_config.print()
print("\n")
a_config = a.Allison_configuration
a_config.print()

In [ ]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
config = mp.configuration()

In [ ]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')

In [ ]:
melt

In [ ]:
sample = "PI032-04-01"
FeO_initial = 15
Fo_host = 0.6
Kd = 0.29
P_bar = 1e3
T_K = 1400
inclusion = melt.loc[sample,:].copy()
# inclusion["H2O"] = 5


In [ ]:
s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
s.get_composition()

In [ ]:
for idx in range(20):

    inclusion = melt.iloc[idx,:].copy()
    s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
    model = "IaconoMarziano"
    VC = v.calculate_saturation_pressure(s, temperature=T_K, model=model).result
    IM, xim = im.calculate_saturation(inclusion, T_K, output="both")
    A, x = a.calculate_saturation(inclusion, T_K, output="both")
    string = f"IM: {IM:>5.0f}, A: {A:>5.0f}"
    print(f"VESIcal: {VC:<10.0f} {string:>25}, delta: {(VC - IM): >3.0f} bar, Xfl-IM: {xim: >4.2f}, Xfl-A: {x: .2f}")

In [ ]:
model = "IaconoMarzianoWater"

p = np.arange(1,  1e4, 5e1)
fig, ax = plt.subplots(figsize=(10, 10))

plt.plot(p, [a.h2o.calculate_solubility(inclusion, P_bar=i, T_K=T_K) for i in p])
plt.plot(p, [v.calculate_dissolved_volatiles(s, temperature=T_K-273.15, pressure=i, model=model).result for i in p], label="VC")

plt.legend()
plt.show()